<h1>Делаем модель нейросети для распознавания рукописных цифр</h1>

https://www.youtube.com/watch?v=QogFIWa1YMg&list=PLA0M1Bcd0w8ynD1umfubKq1OBYRXhXkmH&index=7

In [2]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

In [6]:
import tensorflow as tf
from tensorflow.keras.datasets import mnist
from tensorflow.keras.utils import to_categorical

<h2>Выгрузка данных</h2>

In [4]:
(x_train, y_train), (x_test, y_test) = mnist.load_data()

11490434/11490434 [==============================] - 2s 0us/step


In [7]:
# Нормируем данные от [0, 1]
x_train = x_train / 255
x_test = x_test / 255

# Вытягиваем в один вектор
x_train = tf.reshape(tf.cast(x_train, tf.float32), [-1, 28 * 28])
x_test = tf.reshape(tf.cast(x_test, tf.float32), [-1, 28 * 28])

In [8]:
y_train = to_categorical(y_train, 10)

<h2>Создание полносвязной нейронной сети</h2>

In [9]:
class DenseNN(tf.Module):
    def __init__(self, outputs, activate='relu'):
        super().__init__()
        self.outputs = outputs
        self.activate = activate
        self.fl_init = False

    def __call__(self, x):
        if not self.fl_init:
            self.w = tf.random.truncated_normal((x.shape[-1], self.outputs), stddev=0.1, name='w')
            self.b = tf.zeros([self.outputs], dtype=tf.float32, name='b')

        self.w = tf.Variable(self.w)
        self.b = tf.Variable(self.b)

        self.fl_init = True

        y = x @ self.w + self.b

        if self.activate == 'relu':
            return tf.nn.relu(y)
        elif self.activate == 'softmax':
            return tf.nn.softmax(y)

        return y

In [10]:
# Ининциализация слоев
layer_1 = DenseNN(128)
layer_2 = DenseNN(10, activate='softmax')


def model_predict(x):
    '''Пропускает х через два слоя и возращает предсказание'''
    y = layer_1(x)
    y = layer_2(y)
    return y

In [15]:
# Задаем функцию потерь
cross_entropy = lambda y_true, y_pred: tf.reduce_mean(tf.losses.categorical_crossentropy(y_true, y_pred))
opt = tf.optimizers.legacy.Adam(learning_rate=0.001)

BATCH_SIZE = 32
EPOCHS = 2
TOTAL = x_train.shape[0]

train_dataset = tf.data.Dataset.from_tensor_slices((x_train, y_train))
train_dataset = train_dataset.shuffle(buffer_size=1024).batch(BATCH_SIZE)

In [16]:
for n in range(EPOCHS):
    loss = 0
    for x_batch, y_batch in train_dataset:
        with tf.GradientTape() as tape:
            f_loss = cross_entropy(y_batch, model_predict(x_batch))

        loss += f_loss
        grads = tape.gradient(f_loss, [layer_1.trainable_variables, layer_2.trainable_variables])
        opt.apply_gradients(zip(grads[0], layer_1.trainable_variables))
        opt.apply_gradients(zip(grads[1], layer_2.trainable_variables))

    print(loss.numpy())

391.7502
401.14886


In [14]:
y = model_predict(x_test)
y2 = tf.argmax(y, axis=1).numpy()
acc = len(y_test[y_test == y2]) / y_test.shape[0] * 100
print(acc)

96.75


In [18]:
y = model_predict(x_test)
y2 = tf.argmax(y, axis=1).numpy()
acc = len(y_test[y_test == y2]) / y_test.shape[0] * 100
print(acc)

96.92
